Importacao das bibliotecas

In [50]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models 
import matplotlib.pyplot as plt
import numpy as np

cuDNN para otimizar automaticamente as configurações.

In [51]:
torch.backends.cudnn.benchmark = True

Preparacao, importacao e transformacao dos dados para utilizacao no modelo

In [52]:
transforms = transforms.Compose([
    transforms.Resize((224,224)), # ResNet-50 exige as dimensoes 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalização padrão do ImageNet
])

Criacao dos datasets de treino e teste

In [53]:
train_dataset = datasets.CIFAR10('../data', train=True, download=True, transform=transforms)
test_dataset = datasets.CIFAR10('../data', train=False, download=True, transform=transforms)

Files already downloaded and verified
Files already downloaded and verified


Criacao dos DataLoaders

In [54]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle= True, num_workers=14)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle= False, num_workers=14)

Carregamento do modelo ResNet-50 ja com o pre treino de avaliacao de imagens

In [55]:
model = models.resnet50(pretrained=True)

# Congelar as camadas convulacionais, para que nao sejam treinadas
for param in model.parameters():
    param.requires_grad = False


#Substituir a camada final para o CIFAR-10 (10 Classes)
model.fc = nn.Linear(model.fc.in_features, 10)

# Usar a gpu para execucao do modelo, caso disponivel, se nao ira usar a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Definindo a funcao de perda e o otimizador

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)

Funcao para visualizacao da imagem

In [57]:
def imshow(img):
    img = img / 2 + 0.5
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1,2,0)))
    plt.show()

Treinamento do Modelo

In [58]:
def train(model, train_loader, criterion, optimizer, device, num_epochs = 10):
    model.train() # Iniciar o modo treinamento do modelo
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            #Zerar os gradientes acumulados
            optimizer.zero_grad()

            #Forward Pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)


            #Backward pass e otimizacao 
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoca [{epoch+1}], Perda: {running_loss/len(train_loader)}")

train(model, train_loader, criterion, optimizer, device, num_epochs=10)

Epoca [1], Perda: 0.8861971810764196


KeyboardInterrupt: 

Avaliacao do Modelo

In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Acurácia no conjunto de teste: {accuracy:.2f}%')

# Avaliar o modelo
evaluate(model, test_loader, device)

Visualizar Previsoes

In [ ]:
def visualize_predictions(model, test_loader, device);
    data_iter = iter(test_loader)
    images, labels = next(data_iter)

    #Fazer  previsoes
    outputs = model(images.to(device))
    _, predicted = torch.max(outputs.data, 1)

    #Exibir as imgagens da previsao 
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i in range(5):
        ax = axes[i]
        ax.imshow(images[i].permute(1, 2, 0))
        ax.set_title(f"Pred: {predicted[i]}, Real: {labels[i]}")
        ax.axis('off')
    plt.show()

visualize_predictions(model, test_loader, device)